##Setting up the environment
###Importing libraries:

* os: This module provides a way to use operating system dependent functionality, like reading environment variables.
* OpenAI: This is the official OpenAI Python client, used to interact with OpenAI's API.
* requests: A popular library for making HTTP requests in Python.
* json: Used for parsing JSON data, which is common in API responses.
* logging: Provides a flexible framework for generating log messages in Python.
* getpass: Allows secure password prompts where the input is not displayed on the screen.


###Setting up logging:

* We use `logging.basicConfig()` to configure the logging system. The `level=logging.INFO` argument sets the threshold for logging messages to INFO level and above.
* We create a logger object named logger that we'll use throughout our script to log important information and errors.



This setup ensures we have all necessary tools to interact with APIs, handle data, and track any issues that might occur during execution.

In [6]:
!pip install openai
import os
from openai import OpenAI
import requests
import json
import logging
from getpass import getpass

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("Libraries imported and logging set up successfully.")

Libraries imported and logging set up successfully.


##Secure API key handling
###Secure API key input:

* We use `getpass()` to prompt the user for their API keys. This function hides the input, making it more secure than using regular input().
* This approach is safer than hardcoding API keys in your script, which could accidentally be shared or exposed.


###Setting environment variables:

* We use `os.environ` to set environment variables for both API keys.
* Environment variables are a secure way to store sensitive information, as they're not part of your code and are only accessible within the current process.


###Initializing the OpenAI client:

* We create an instance of the OpenAI client using the API key we just set.
* Using `os.getenv("OPENAI_API_KEY")` retrieves the API key from the environment variables.



This method ensures that your API keys are handled securely and are readily available for use in your script.

In [7]:
# Cell 2: Secure API key handling

# Securely input API keys
openai_api_key = getpass("Enter your OpenAI API key: ")
llumo_api_key = getpass("Enter your Llumo API key: ")

# Set environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['LLUMO_API_KEY'] = llumo_api_key

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("API keys securely set and OpenAI client initialized.")

Enter your OpenAI API key: ··········
Enter your Llumo API key: ··········
API keys securely set and OpenAI client initialized.


##Define Llumo compression function
###Function definition:

* We define a function `compress_with_llumo` that takes a text input and an optional topic.


###API setup:

* We retrieve the Llumo API key from environment variables.
* We set the API endpoint and prepare headers for the HTTP request.


###Payload preparation:

* We create a payload dictionary with the input text.
* If a topic is provided, we add it to the payload.


###API request:

* We use `requests.post()` to send a POST request to the Llumo API.
`response.raise_for_status()` will raise an exception for HTTP errors.


###Response parsing:

* We parse the JSON response and extract the compressed text and token counts.
* We calculate the compression percentage.


###Error handling:

* We use a try-except block to catch potential errors:

 * JSON decoding errors
 * Request exceptions
 * Unexpected response structure


* If an error occurs, we log it and return the original text with failure indicators.


### Return values:

* The function returns a tuple containing:

 * Compressed text (or original if compression failed)
 * Success boolean
 * Compression percentage
 * Initial token count
 * Final token count





This function encapsulates the entire process of interacting with the Llumo API for text compression, including error handling and result processing.

In [3]:
def compress_with_llumo(text):
    # Retrieve the Llumo API key from environment variables
    LLUMO_API_KEY = os.getenv('LLUMO_API_KEY')

    # Define the Llumo API endpoint for text compression
    LLUMO_ENDPOINT = "https://app.llumo.ai/api/compress"

    # Set the headers for the API request
    headers = {
        "Content-Type": "application/json",  # Specify the content type as JSON
        "Authorization": f"Bearer {LLUMO_API_KEY}"  # Authorization header with the API key
    }

    # Create the payload for the API request with the provided text
    payload = {"prompt": text}

    try:
        # Make a POST request to the Llumo API with the payload and headers
        response = requests.post(LLUMO_ENDPOINT, json=payload, headers=headers)
        response.raise_for_status()

        # Parse the JSON response from the API
        result = response.json()
        data = json.loads(result['data']['data'])

        # Extract compressed text and token information
        compressed_text = data.get('compressedPrompt', text)
        initial_tokens = data.get('initialTokens', 0)
        final_tokens = data.get('finalTokens', 0)

        # Calculate the compression percentage
        compression_percentage = ((initial_tokens - final_tokens) / initial_tokens) * 100 if initial_tokens else 0

        # Return the compressed text, success flag, compression percentage, and token counts
        return compressed_text, True, compression_percentage, initial_tokens, final_tokens
    except Exception as e:
        print(f"Error compressing text: {str(e)}")
        return text, False, 0, 0, 0


##Define example prompt and test without compression
###Defining the prompt:

* We create a detailed prompt about photosynthesis. This serves as our example text for compression.


###Testing without compression:

* We use the OpenAI client to send a request to the GPT-3.5-turbo model.
* The messages parameter follows the chat format:

 * A system message sets the AI's role.
 * A user message contains our prompt.




###Displaying results:

* We print the AI's response to the prompt.
* We also print the total number of tokens used, which is important for understanding API usage and costs.

This cell demonstrates how the API would typically be used without any compression, providing a baseline for comparison.



In [8]:
# Cell 4: Define example prompt and test without compression

# Example prompt
prompt = """
Context: Flexicab: The Smart Rickshaw Management PlatformFlexicab is a revolutionary SaaS platform designed specifically for Rickshaw drivers and owners in bustling urban environments. It addresses the unique challenges faced by this vital mode of transportation, empowering them with modern tools to optimize their operations and increase their earnings.**Key Features:*** **Route Optimization:** Flexicab leverages real-time traffic data and historical ride patterns to suggest the most efficient routes for each trip, minimizing travel time and fuel consumption. This feature helps drivers maximize their earnings by completing more rides within a given timeframe.* **Fare Calculation & Management:** Flexicab automatically calculates fares based on distance, time, and other factors, ensuring transparency and accuracy for both drivers and passengers. It also integrates with popular payment gateways, allowing for seamless digital transactions and reducing the risk of cash handling.* **Customer Management:** Flexicab provides a platform for drivers to manage their customer base, track ride history, and build loyalty through personalized offers and promotions. This helps drivers attract repeat customers and increase their overall revenue.* **Fleet Management:** For Rickshaw owners with multiple vehicles, Flexicab offers a comprehensive fleet management system. This includes vehicle tracking, maintenance scheduling, driver performance monitoring, and real-time communication tools to ensure efficient operations and safety.* **Data Analytics & Insights:** Flexicab provides valuable insights into driver performance, customer behavior, and market trends. This data can be used to optimize pricing strategies, identify high-demand areas, and make informed business decisions.**Benefits:*** **Increased Earnings:** By optimizing routes, managing fares effectively, and attracting more customers, Flexicab helps drivers significantly increase their earnings.* **Improved Efficiency:** Flexicab streamlines operations, reducing wasted time and effort, allowing drivers to focus on providing excellent service.* **Enhanced Safety:** The platform's features, such as vehicle tracking and driver performance monitoring, contribute to a safer environment for both drivers and passengers.* **Digital Transformation:** Flexicab empowers Rickshaw drivers and owners to embrace digital technology, making their businesses more modern and competitive.**Target Audience:*** Rickshaw drivers* Rickshaw owners* Transportation companies operating Rickshaws* City governments and transportation authorities looking to improve urban mobility and support sustainable transportation options.**Flexicab is more than just a platform; it's a comprehensive solution that empowers Rickshaw drivers and owners to thrive in the modern urban landscape.
Query: How does Flexicab help Rickshaw drivers increase their earnings?
Instructions:You are a helpful and informative chatbot designed to answer questions about Flexicab, a SaaS platform for Rickshaw drivers and owners. Use the provided context to answer the user's query in a comprehensive and engaging manner. **Response Format:*** Begin your response with a clear and concise answer to the user's question.* Provide additional details and explanations to support your answer, drawing from the provided context.* Use bullet points or numbered lists to organize information and make it easier to read.* If applicable, include specific examples or scenarios to illustrate your points.* End your response with a friendly and helpful tone, encouraging further interaction.**
"""



print("Without Llumo Compression:")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)
print(response.choices[0].message.content)
print(f"Tokens used: {response.usage.total_tokens}\n")

Without Llumo Compression:
**How does Flexicab help Rickshaw drivers increase their earnings?**

Flexicab empowers Rickshaw drivers to boost their earnings through various key features and benefits tailored to enhance their efficiency and profitability:

1. **Route Optimization**: By utilizing real-time traffic data and historical ride patterns, Flexicab suggests the most efficient routes for each trip. This minimizes travel time and fuel consumption, enabling drivers to complete more rides within a given timeframe and increase their daily earnings.

2. **Fare Calculation & Management**: The platform automatically calculates fares based on distance, time, and other factors, ensuring transparency for both drivers and passengers. This accurate fare calculation system eliminates any uncertainty around pricing, allowing drivers to maximize their earning potential on each trip.

3. **Customer Management**: Flexicab provides drivers with tools to manage their customer base, track ride histor

##Test with Llumo compression
###Applying Llumo compression:

* We call compress_with_llumo() with our example prompt.
* The function returns multiple values, which we unpack into separate variables.


###Checking compression success:

* We use an if statement to check if compression was successful.
* If successful, we print compression statistics: percentage, initial and final token counts.


###Using the compressed prompt:

* If compression succeeded, we use the compressed prompt in our API call to GPT-3.5-turbo.
* We use the same message structure as before, but with the compressed prompt.


###Displaying results:

* We print the AI's response to the compressed prompt.
* We print the number of tokens used with the compressed prompt.


###Handling compression failure:

* If compression fails, we print a message indicating this.
* In a real application, you might want to fall back to using the original prompt in this case.



This cell demonstrates the full process of compressing a prompt with Llumo and using it with the OpenAI API. By comparing the results and token usage with the previous cell, you can see the potential benefits of using Llumo compression in terms of token efficiency and cost savings.

In [9]:
# Cell 5: Test with Llumo compression

print("With Llumo Compression:")
compressed_prompt, success, compression_percentage, initial_tokens, final_tokens = compress_with_llumo(prompt)

if success:
    print(f"Compression achieved: {compression_percentage:.2f}%")
    print(f"Initial tokens: {initial_tokens}, Final tokens: {final_tokens}")

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": compressed_prompt}
        ]
    )
    print(response.choices[0].message.content)
    print(f"Tokens used: {response.usage.total_tokens}")
else:
    print("Compression failed. Using original prompt.")
    # Use the original prompt if compression fails

With Llumo Compression:
Compression achieved: 58.64%
Initial tokens: 631, Final tokens: 261
To help Rickshaw drivers increase their earnings, Flexicab provides a range of features and benefits that optimize their operations and enhance their overall performance. Here are some ways in which Flexicab helps drivers improve their earnings:

1. **Route Optimization**: Flexicab helps drivers by optimizing their routes, suggesting the most efficient paths to maximize the number of rides they can complete in a specific time frame. This ensures that drivers can cater to more customers and earn more during their working hours.

2. **Enhanced Communication and Data Analysis**: By providing real-time communication tools and insights into driver and industry trends, Flexicab enables drivers to make informed decisions. This helps them identify high-demand areas and adjust their services accordingly to attract more customers.

3. **Seamless Payment Transactions**: Flexicab facilitates seamless paymen